In [1]:
import os
os.chdir("../")

In [2]:
import os
import tensorflow as tf
from tensorflow.python import keras
from bert import BertModelLayer
from bert.loader import StockBertConfig, load_stock_weights

In [3]:
model_dir = "/Users/naval/Desktop/Naval/CBM/BERT-NER-master/bert/multi_cased_L-12_H-768_A-12"

bert_config_file = os.path.join(model_dir, "bert_config.json")
bert_ckpt_file   = os.path.join(model_dir, "bert_model.ckpt")

with tf.io.gfile.GFile(bert_config_file, "r") as reader:
  stock_params = StockBertConfig.from_json_string(reader.read())
  bert_params  = stock_params.to_bert_model_layer_params()

bert = BertModelLayer.from_params(bert_params, name="bert")

In [4]:
bert_params

{'initializer_range': 0.02,
 'max_position_embeddings': 512,
 'hidden_size': 768,
 'vocab_size': 119547,
 'use_token_type': True,
 'use_position_embeddings': True,
 'token_type_vocab_size': 2,
 'hidden_dropout': 0.1,
 'adapter_size': None,
 'adapter_activation': 'gelu',
 'adapter_init_scale': 0.001,
 'num_heads': 12,
 'size_per_head': None,
 'query_activation': None,
 'key_activation': None,
 'value_activation': None,
 'attention_dropout': 0.1,
 'negative_infinity': -10000.0,
 'intermediate_size': 3072,
 'intermediate_activation': 'gelu',
 'num_layers': 12,
 'out_layer_ndxs': None}

In [5]:
def create_initializer(initializer_range=0.02):
  """Creates a `truncated_normal_initializer` with the given range."""
  return tf.compat.v1.initializers.truncated_normal(stddev=initializer_range)

In [6]:
tf.keras.layers.Dense

tensorflow.python.keras.layers.core.Dense

In [9]:
from tensorflow.python import keras

max_seq_len = 128
input_ids      = keras.layers.Input(shape=(max_seq_len,), dtype='int32', name="input_ids")
token_type_ids = keras.layers.Input(shape=(max_seq_len,), dtype='int32', name="token_type_ids")

sequence_output = bert([input_ids, token_type_ids])  # [batch_size, max_seq_len, hidden_size]

first_token_tensor = tf.squeeze(sequence_output[:, 0:1, :], axis=1)

pooled_output = tf.keras.layers.Dense(
            first_token_tensor,
            bert_params.hidden_size,
            activation="tanh",
            kernel_initializer=create_initializer(bert_params.initializer_range))

TypeError: __init__() got multiple values for argument 'activation'

In [8]:
tf.tanh

<function tensorflow.python.ops.gen_math_ops.tanh(x, name=None)>

In [4]:
from tensorflow.python import keras

max_seq_len = 128
input_ids      = keras.layers.Input(shape=(max_seq_len,), dtype='int32', name="input_ids")
token_type_ids = keras.layers.Input(shape=(max_seq_len,), dtype='int32', name="token_type_ids")

output = bert([input_ids, token_type_ids])  # [batch_size, max_seq_len, hidden_size]
model = keras.Model(inputs=[input_ids, token_type_ids], outputs=output)

In [5]:
model.build(input_shape=[(None, max_seq_len),
                                 (None, max_seq_len)])

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
bert (BertModelLayer)           (None, 128, 768)     177262848   input_ids[0][0]                  
                                                                 token_type_ids[0][0]             
Total params: 177,262,848
Trainable params: 177,262,848
Non-trainable params: 0
__________________________________________________________________________________________________


In [7]:
output.shape

TensorShape([None, 128, 768])